# hashlib

hashlib is a standard Python library that provides cryptographic hash functions.
A hash function converts arbitrary-length input data into a fixed-size, deterministic, one-way digest.

**Key characteristics:**

* Deterministic (same input → same output)
* One-way (cannot reverse the hash)
* Fixed output size
* Small input change → large output change (avalanche effect)

| Algorithm | Digest Size | Status                             |
| --------- | ----------- | ---------------------------------- |
| MD5       | 128-bit     | ❌ Broken (do not use for security) |
| SHA1      | 160-bit     | ❌ Weak                             |
| SHA224    | 224-bit     | ✅                                  |
| SHA256    | 256-bit     | ✅ Recommended                      |
| SHA384    | 384-bit     | ✅                                  |
| SHA512    | 512-bit     | ✅                                  |
| BLAKE2b   | Variable    | ✅ Fast & secure                    |
| BLAKE2s   | Variable    | ✅                                  |


In [32]:
import hashlib
import secrets
import hmac
import os

In [33]:
print(hashlib.algorithms_available)

{'sm3', 'sha3_512', 'shake_256', 'md5-sha1', 'sha1', 'sha512_256', 'sha3_224', 'blake2s', 'shake_128', 'sha384', 'sha512_224', 'ripemd160', 'sha224', 'blake2b', 'sha256', 'sha3_384', 'md5', 'sha3_256', 'sha512'}


Passwords are validated by comparing a newly computed hash with the stored hash.

There is no decoding or recovery of the original password.


**1. High-level password validation flow**

* User enters a password
* System retrieves the stored salt + hash
* System hashes the input password using the same algorithm and parameters
* The two hashes are compared
* Match → authentication succeeds
* No match → authentication fails

**The plaintext password is never stored and never reconstructed.It is one side hash and not decoded or decryted, it is validated through the stored hash**

What happens on a database breach

**Attackers get:**

    Salt
    Hash

They do not get passwords.

**To crack:**

    They must guess passwords
    Hash each guess
    Compare results
    Repeat millions of times

**This is why:**

    High iteration counts matter
    Unique salts are mandatory

### Passsword Hashing Example

In [34]:
def hash_password(password, salt=None, iterations=200_000):
    if salt is None:
        salt = secrets.token_bytes(32)  # Generate a new random salt
    # Use PBKDF2 with SHA256
    hash_bytes = hashlib.pbkdf2_hmac(
        hash_name='sha256',
        password=password,
        salt=salt,
        iterations=iterations
    )
    return salt, hash_bytes


salt, hashed = hash_password(b'shra123456')
salt.hex(), hashed.hex()

('dd9476d14814a1571aa4197644ad74b27cd06a58dcf96f848a695f0abb14779a',
 '59ae9b7c1d161bce25e398bc618a86bf9e209cffadca95b3d5b5ee9c8f7ad6c0')

In [35]:
def verify_password(stored_salt, stored_hash, input_password, iterations=200_000):
    # Hash the input password with the stored salt
    input_hash = hashlib.pbkdf2_hmac(
        hash_name='sha256',
        password=input_password,
        salt=stored_salt,
        iterations=iterations
    )
    # print(hmac.compare_digest(input_hash, stored_hash))
    return input_hash == stored_hash

In [36]:
is_valid = verify_password(salt, hashed, b'shra123456', iterations=200_000)
is_valid

True

In [37]:
is_valid = verify_password(salt, hashed, b'wrongpassword', iterations=200_000)
is_valid

False

**hmac:** Provides message authentication by combining a secret key with a hash function to verify data integrity and authenticity.

**hashlib:** Generates cryptographic hash digests for data integrity, fingerprinting, and secure password derivation (without authentication by itself).

**secrets:** Generates cryptographically secure random values for tokens, passwords, and sensitive security-related data.